# Build a Chatbot with Langchain and Chorma

In [1]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "./example_data/Badham.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

In [2]:
import textwrap
from IPython.display import display
from IPython.display import Markdown
import os


# using v2rayn
os.environ["HTTP_PROXY"] = "http://127.0.0.1:10809"
os.environ["HTTP_PROXYS"] = "http://127.0.0.1:10809"

# AI model
model = "gemini-1.5-flash"
google_api_key=os.getenv("GEMINI_API_KEY")


def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model=model,
                             google_api_key=google_api_key)

In [4]:
# import google.generativeai as genai
# genai.configure(api_key=google_api_key)
# for m in genai.list_models():
#   if 'embedContent' in m.supported_generation_methods:
#     print(m.name)

In [5]:
# import bs4
from langchain import hub
from langchain_chroma import Chroma
# from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
# from langchain_openai import OpenAIEmbeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# # Load, chunk and index the contents of the blog.
# loader = WebBaseLoader(
#     web_paths=("https://en.wikipedia.org/wiki/Bass_diffusion_model",),
#     bs_kwargs=dict(
#         # parse_only=bs4.SoupStrainer(
#         #     class_=("post-content", "post-title", "post-header")
#         # )
#     ),
# )


# docs = loader.load()

In [8]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits,
                                    embedding=GoogleGenerativeAIEmbeddings(
                                        model="models/text-embedding-004",
                                        google_api_key=google_api_key)
                                    )
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

GoogleGenerativeAIError: Error embedding content: 504 Deadline Exceeded

In [ ]:
to_markdown(rag_chain.invoke("What is this paper about?"))